In [2]:
import os
import pandas as pd
import sqlite3
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
BASE_URL = os.getenv("BASE_URL")

# Create a sample DataFrame (replace this with your actual data loading)
df = pd.read_csv(
    "C:/Users/nazaizah/OneDrive/PycharmProjects/dabtech-llm-workshop/data/members/stats/members_stats.csv"
)

In [3]:
# Connect to SQLite database (creates the file if it doesn't exist)
conn = sqlite3.connect("data.db")

# Save DataFrame to SQL database
df.to_sql("members", conn, if_exists="replace", index=False)

# Verify the data was saved
print("Data saved successfully!")
print(f"Number of rows saved: {len(df)}")

# Close the connection
conn.close()

Data saved successfully!
Number of rows saved: 116


In [ ]:
conn = sqlite3.connect("data.db")
cursor = conn.cursor()
tables = cursor.execute(
    """
    SELECT name FROM sqlite_master 
    WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""
).fetchall()

tables

[('members',)]

In [ ]:
table_name = "members"
columns = cursor.execute(f"PRAGMA table_info({table_name});").fetchall()
columns

[(0, 'name', 'TEXT', 0, None, 0),
 (1, 'current_status', 'TEXT', 0, None, 0),
 (2, 'current_title', 'TEXT', 0, None, 0),
 (3, 'current_company', 'TEXT', 0, None, 0),
 (4, 'first_job_start', 'TEXT', 0, None, 0),
 (5, 'total_years_experience', 'REAL', 0, None, 0),
 (6, 'years_experience_bucket', 'TEXT', 0, None, 0),
 (7, 'highest_degree', 'TEXT', 0, None, 0),
 (8, 'institution', 'TEXT', 0, None, 0),
 (9, 'graduation_year', 'REAL', 0, None, 0),
 (10, 'linkedin_url', 'TEXT', 0, None, 0)]

In [ ]:
df.info()

In [ ]:
db_context = """
DATABASE SCHEMA CONTEXT

Tables and Columns:

1. members
- name (TEXT)
- current_status (TEXT)
- current_title (TEXT)
- current_company (TEXT)
- first_job_start (TEXT)
- total_years_experience (FLOAT)
- years_experience_bucket (TEXT)
- highest_degree (TEXT)
- institution (TEXT)
- graduation_year (FLOAT)
- linkedin_url (TEXT)
"""

In [ ]:
# Connect to the database and run a query
conn = sqlite3.connect("data.db")

# Example query - get all members with their current status
query = "SELECT * FROM members LIMIT 10"
result = pd.read_sql_query(query, conn)
print(result)

# Close the connection
conn.close()

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=GITHUB_TOKEN, base_url=BASE_URL)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

In [ ]:
import os, requests

TBA_KEY = "Vjrkd6SEV9noCzSkjezkexNMJKIcYnyhBOKGb8gx71XhcWHmY0WYN48p023KBzzA"  # os.environ.get("TBA_KEY")  # get your key from your TBA account settings
headers = {"X-TBA-Auth-Key": TBA_KEY}

# Example: list 2025 events
events = requests.get(
    "https://www.thebluealliance.com/api/v3/events/2025", headers=headers
).json()
print(len(events), "events. First 3 names:", [e["name"] for e in events[:3]])

# Example: matches for Team 5715 at a specific event (e.g., 2025isde1)
team_key = "frc5715"
event_key = "2025isde1"
matches = requests.get(
    f"https://www.thebluealliance.com/api/v3/team/{team_key}/event/{event_key}/matches",
    headers=headers,
).json()
for m in matches:
    print(m["key"], m["comp_level"], "time:", m.get("time"))

In [ ]:
import os
import requests

# TBA_KEY = os.environ.get("TBA_KEY", "your_key_here")
HEADERS = {
    "X-TBA-Auth-Key": TBA_KEY,
    "Accept": "application/json",
    "User-Agent": "my-app/1.0",
}


def find_team_by_name(search_str, max_pages=10):
    search_lower = search_str.lower()
    for page in range(max_pages):
        url = f"https://www.thebluealliance.com/api/v3/teams/{page}"
        resp = requests.get(url, headers=HEADERS)
        resp.raise_for_status()
        teams = resp.json()
        if not teams:
            break
        for team in teams:
            # check nickname, name and optionally other fields
            if (
                team.get("nickname", "").lower().find(search_lower) >= 0
                or team.get("name", "").lower().find(search_lower) >= 0
            ):
                # found a match
                return team
    return None


# example usage:
result = find_team_by_name("dabburiya")
if result:
    print("Found team:", result["team_number"], result["nickname"])
else:
    print("Team not found.")

In [ ]:
url = f"https://www.thebluealliance.com/api/v3/team/frc5715/event/2024iscmp/matches"
matches = requests.get(url, headers=headers).json()
matches

In [ ]:
for match in matches:
    match.pop("score_breakdown", None)
matches

In [ ]:
print(
    "```python\nimport plotly.graph_objects as go\n\ndata = {\n    'institution': [\n        'Technion - Israel Institute of Technology',\n        'University of Haifa',\n        'Tel Aviv University',\n        'None',\n        'Jerusalem Multidisciplinary College',\n        'The Max Stern Yezreel Valley College',\n        'Kinneret Academic College',\n        'Ben-Gurion University of the Negev',\n        'Bar-Ilan University',\n        'The Open University of Israel'\n    ],\n    'member_count': [35, 22, 11, 8, 5, 4, 4, 4, 3, 2]\n}\n\nfig = go.Figure(data=[go.Bar(\n    x=data['institution'],\n    y=data['member_count'],\n)])\n\nfig.update_layout(\n    title='Member Count by Institution',\n    xaxis_title='Institution',\n    yaxis_title='Member Count',\n    xaxis_tickangle=-45\n)\n\n```"
)

In [ ]:
import plotly.graph_objects as go

code = "import plotly.graph_objects as go\n\ndata = {\n    'institution': [\n        'Technion - Israel Institute of Technology',\n        'University of Haifa',\n        'Tel Aviv University',\n        'None',\n        'Jerusalem Multidisciplinary College',\n        'The Max Stern Yezreel Valley College',\n        'Kinneret Academic College',\n        'Ben-Gurion University of the Negev',\n        'Bar-Ilan University',\n        'The Open University of Israel'\n    ],\n    'member_count': [35, 22, 11, 8, 5, 4, 4, 4, 3, 2]\n}\n\nfig = go.Figure(data=[go.Bar(\n    x=data['institution'],\n    y=data['member_count'],\n)])\n\nfig.update_layout(\n    title='Member Count by Institution',\n    xaxis_title='Institution',\n    yaxis_title='Member Count',\n    xaxis_tickangle=-45\n)\n\n"
local_vars = {}
exec(code, {"go": go}, local_vars)
fig = local_vars.get("fig")
fig